In [1]:
import pandas as pd
from src.training.bert_pipeline import TrainingBertPipeline
import logging
import torch
import os

In [2]:
df = pd.read_csv("data/aes_dataset_5k_clean.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5363 entries, 0 to 5362
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   question          4859 non-null   object 
 1   reference_answer  5363 non-null   object 
 2   answer            5363 non-null   object 
 3   score             5363 non-null   float64
 4   normalized_score  5363 non-null   float64
 5   multibert_length  5363 non-null   int64  
 6   dataset           5363 non-null   object 
 7   dataset_num       5363 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 335.3+ KB


In [3]:
df['dataset'].value_counts()

dataset
sag               2558
analisis_essay    2162
stita              333
cunlp              171
sci                139
Name: count, dtype: int64

In [4]:
# Check if the first file exists
df_result = None
if os.path.exists("experiments/results/results.csv"):
    df_result = pd.read_csv("experiments/results/results.csv")
    print(df_result['config_id'].iloc[-1])
else:
    print("File 'results.csv' does not exist.")

11


In [5]:
batch_sizes = [4, 8]
overlappings = [256]
epochs_list = [5, 10]
learning_rates = [1e-5, 2e-5, 5e-5]
idx = (df_result['config_id'].iloc[-1] + 1) if df_result is not None and not df_result.empty else 0  # index untuk setiap kombinasi
ROOT_DIR = os.getcwd()

In [6]:
# model = [
#     ("bert_length", "bert-base-uncased"),
#     ("indobert_length", "indobenchmark/indobert-base-p1"),
#     ("albert_length", "albert-base-v1"),
#     ("indoalbert_length", "indobenchmark/indobert-lite-base-p2"),
#     ("longformer_length", "allenai/longformer-base-4096"),
#     ("multibert_length", "google-bert/bert-base-multilingual-uncased")
# ]

In [7]:
for batch_size in batch_sizes:
    for overlapping in overlappings:
        for num_epochs in epochs_list:
            for lr in learning_rates:
                results = []
                results_epoch = []
                df_result1 = None
                # Check if the second file exists
                if os.path.exists("experiments/results/results_epoch.csv"):
                    df_result1 = pd.read_csv("experiments/results/results_epoch.csv")
                    print(max(df_result1['valid_qwk']))
                else:
                    print("File 'results_epoch.csv' does not exist.")
                config = {
                    "df": df,
                    "model_name": "google-bert/bert-base-multilingual-uncased",
                    "overlapping": overlapping,
                    "batch_size": batch_size,
                    "learning_rate": lr,
                    "epochs": num_epochs,
                    "config_id": idx,
                    "max_seq_len": 512,
                    "col_length": "multibert_length",
                    "best_valid_qwk": max(df_result1['valid_qwk']) if df_result1 is not None and not df_result1.empty else float("-inf")
                }

                logging.info(
                    f"Running configuration: config_id={idx}, model_name={config['model_name']}, batch_size={batch_size}, "
                    f"max_seq_length={config['max_seq_len']}, overlapping={overlapping}, epochs={num_epochs}, learning_rate={lr}"
                )
                
                print(
                    f"\nRunning configuration: config_id={idx}, model_name={config['model_name']}, batch_size={batch_size}, "
                    f"max_seq_length={config['max_seq_len']}, overlapping={overlapping}, epochs={num_epochs}, learning_rate={lr}"
                )
                
                try:
                    pipeline = TrainingBertPipeline(config, results, results_epoch)
                    pipeline.run_training()

                    # Save results
                    # Dapatkan root project
                    results_path = os.path.join(ROOT_DIR, "experiments/results/results.csv")
                    results_epoch_path = os.path.join(ROOT_DIR, "experiments/results/results_epoch.csv")
                    TrainingBertPipeline.save_csv(results, results_path)
                    TrainingBertPipeline.save_csv(results_epoch, results_epoch_path)
                except Exception as e:
                    logging.error(f"Error in config_id={idx}: {str(e)}")
                    print(f"Error in config_id={idx}: {str(e)}")
                    torch.cuda.empty_cache()
                finally:
                    # Clear GPU memory after every configuration
                    del pipeline.model
                    del pipeline.tokenizer
                    del pipeline.optimizer
                    torch.cuda.empty_cache()

                idx += 1

0.9251343231432688

Running configuration: config_id=12, model_name=google-bert/bert-base-multilingual-uncased, batch_size=4, max_seq_length=512, overlapping=256, epochs=5, learning_rate=1e-05
split dataset run...
create dataset run...
max len 512
max len 512
max len 512
create dataloader run...
====== Training Epoch 1/5 ======


Token indices sequence length is longer than the specified maximum sequence length for this model (1004 > 512). Running this sequence through the model will result in indexing errors
c:\Users\User\Documents\Code\env\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Train Loss: 0.0498, Train QWK: 0.6411, Train Pearson: 0.7456
Validation Loss: 0.0285, Validation QWK: 0.8735, Validation Pearson: 0.8895
====== Training Epoch 2/5 ======
Train Loss: 0.0274, Train QWK: 0.7249, Train Pearson: 0.8641
Validation Loss: 0.0198, Validation QWK: 0.9088, Validation Pearson: 0.9114
====== Training Epoch 3/5 ======
Train Loss: 0.0201, Train QWK: 0.7732, Train Pearson: 0.9021
Validation Loss: 0.0210, Validation QWK: 0.9045, Validation Pearson: 0.9163
====== Training Epoch 4/5 ======
Train Loss: 0.0154, Train QWK: 0.7948, Train Pearson: 0.9254
Validation Loss: 0.0188, Validation QWK: 0.9114, Validation Pearson: 0.9146
====== Training Epoch 5/5 ======
Train Loss: 0.0118, Train QWK: 0.8279, Train Pearson: 0.9436
Validation Loss: 0.0226, Validation QWK: 0.9035, Validation Pearson: 0.9174
Test Loss: 0.0215, Test QWK: 0.9091, Test Pearson: 0.9221
0.9251343231432688

Running configuration: config_id=13, model_name=google-bert/bert-base-multilingual-uncased, batch_size=4,

Token indices sequence length is longer than the specified maximum sequence length for this model (1004 > 512). Running this sequence through the model will result in indexing errors


Train Loss: 0.0468, Train QWK: 0.6672, Train Pearson: 0.7572
Validation Loss: 0.0278, Validation QWK: 0.8755, Validation Pearson: 0.8948
====== Training Epoch 2/5 ======
Train Loss: 0.0299, Train QWK: 0.7236, Train Pearson: 0.8520
Validation Loss: 0.0272, Validation QWK: 0.8812, Validation Pearson: 0.8985
====== Training Epoch 3/5 ======
Train Loss: 0.0229, Train QWK: 0.7477, Train Pearson: 0.8885
Validation Loss: 0.0248, Validation QWK: 0.9000, Validation Pearson: 0.8968
====== Training Epoch 4/5 ======
Train Loss: 0.0180, Train QWK: 0.7776, Train Pearson: 0.9136
Validation Loss: 0.0250, Validation QWK: 0.8815, Validation Pearson: 0.9053
====== Training Epoch 5/5 ======
Train Loss: 0.0182, Train QWK: 0.7899, Train Pearson: 0.9121
Validation Loss: 0.0266, Validation QWK: 0.8924, Validation Pearson: 0.9042
Test Loss: 0.0242, Test QWK: 0.9036, Test Pearson: 0.9147
0.9251343231432688

Running configuration: config_id=14, model_name=google-bert/bert-base-multilingual-uncased, batch_size=4,

...

split dataset run...
create dataset run...
max len 512
max len 512
max len 512
create dataloader run...
====== Training Epoch 1/10 ======
Train Loss: 0.0439, Train QWK: 0.6604, Train Pearson: 0.7728
Validation Loss: 0.0404, Validation QWK: 0.8476, Validation Pearson: 0.8853
====== Training Epoch 2/10 ======
Train Loss: 0.0256, Train QWK: 0.7443, Train Pearson: 0.8733
Validation Loss: 0.0213, Validation QWK: 0.8998, Validation Pearson: 0.9065
====== Training Epoch 3/10 ======
Train Loss: 0.0194, Train QWK: 0.7809, Train Pearson: 0.9054
Validation Loss: 0.0194, Validation QWK: 0.9102, Validation Pearson: 0.9153
====== Training Epoch 4/10 ======
Train Loss: 0.0151, Train QWK: 0.8067, Train Pearson: 0.9268
Validation Loss: 0.0211, Validation QWK: 0.9044, Validation Pearson: 0.9105
====== Training Epoch 5/10 ======
Train Loss: 0.0123, Train QWK: 0.8170, Train Pearson: 0.9411
Validation Loss: 0.0262, Validation QWK: 0.8884, Validation Pearson: 0.9129
====== Training Epoch 6/10 ======
Train L

Token indices sequence length is longer than the specified maximum sequence length for this model (1004 > 512). Running this sequence through the model will result in indexing errors


Train Loss: 0.0462, Train QWK: 0.6611, Train Pearson: 0.7605
Validation Loss: 0.0413, Validation QWK: 0.8557, Validation Pearson: 0.8882
====== Training Epoch 2/10 ======
Train Loss: 0.0264, Train QWK: 0.7516, Train Pearson: 0.8700
Validation Loss: 0.0238, Validation QWK: 0.8947, Validation Pearson: 0.9029
====== Training Epoch 3/10 ======
Train Loss: 0.0203, Train QWK: 0.7737, Train Pearson: 0.9014
Validation Loss: 0.0230, Validation QWK: 0.8985, Validation Pearson: 0.8998
====== Training Epoch 4/10 ======
Train Loss: 0.0172, Train QWK: 0.7906, Train Pearson: 0.9171
Validation Loss: 0.0217, Validation QWK: 0.9009, Validation Pearson: 0.9025
====== Training Epoch 5/10 ======
Train Loss: 0.0136, Train QWK: 0.8225, Train Pearson: 0.9349
Validation Loss: 0.0266, Validation QWK: 0.8844, Validation Pearson: 0.9039
====== Training Epoch 6/10 ======
Train Loss: 0.0104, Train QWK: 0.8247, Train Pearson: 0.9508
Validation Loss: 0.0206, Validation QWK: 0.9048, Validation Pearson: 0.9110
====== T

Token indices sequence length is longer than the specified maximum sequence length for this model (1004 > 512). Running this sequence through the model will result in indexing errors


split dataset run...
create dataset run...
max len 512
max len 512
max len 512
create dataloader run...
====== Training Epoch 1/10 ======
Train Loss: 0.0478, Train QWK: 0.6486, Train Pearson: 0.7548
Validation Loss: 0.0518, Validation QWK: 0.8405, Validation Pearson: 0.8969
====== Training Epoch 2/10 ======
Train Loss: 0.0291, Train QWK: 0.7376, Train Pearson: 0.8558
Validation Loss: 0.0282, Validation QWK: 0.8879, Validation Pearson: 0.9014
====== Training Epoch 3/10 ======
Train Loss: 0.0245, Train QWK: 0.7475, Train Pearson: 0.8800
Validation Loss: 0.0273, Validation QWK: 0.8725, Validation Pearson: 0.8769
====== Training Epoch 4/10 ======
Train Loss: 0.0194, Train QWK: 0.7790, Train Pearson: 0.9060
Validation Loss: 0.0258, Validation QWK: 0.8704, Validation Pearson: 0.8828
====== Training Epoch 5/10 ======
Train Loss: 0.0154, Train QWK: 0.8064, Train Pearson: 0.9257
Validation Loss: 0.0270, Validation QWK: 0.8817, Validation Pearson: 0.8918
====== Training Epoch 6/10 ======
Train L